# Phase picking

Now we have detected some earthquakes we should do something with them.  We will discuss location of earthquakes in
[the next notebook](3_Earthquake_Location.ipynb), but classical earthquake detection and analysis requires the
accurate identification of seismic phase arrivals.  In this notebook we will look at some picking stratergies and
think about a few automatic pickers implemented in Obspy.

First, lets take an example of an earthquake in the GeoNet catalog and plot the waveforms with the picks overlain.

**Note:** We need to use a fully interactive plotting backend for the `SeismicPicker` to work, using the line `%matplotlib` sets us up with the default backend.  You won't see plots in the notebook, instead you will get pop-up windows.

In [1]:
# Notebook set-up
%matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np

from obspy import Stream
from obspy.clients.fdsn import Client

from utilities.plot_event import get_geonet_waveforms, plot_picked

client = Client("GEONET")
cat = client.get_events(eventid="2018p546311")
event = cat[0]
print("Event has {0} picks".format(len(event.picks)))
# We will remove the picks from strong motion sites, they often have poor timing
# This "list-comprehension" overwrite the "picks" attibrute of the event
# with all the picks that were not made on a channel with code starting with "B"
event.picks = [p for p in event.picks 
               if p.waveform_id.channel_code[0] != "B"]

# Lets just look at the earliest picked ten stations.

# Sort the picks by time
event.picks.sort(key=lambda p: p.time)
# Make a list of the station codes for the first ten stations
first_ten_stations = {p.waveform_id.station_code for p in event.picks[0:10]}
# Now overwrite the "picks" attribute with only picks from one of those ten stations
event.picks = [p for p in event.picks if p.waveform_id.station_code in first_ten_stations]

# Now we will get the data that we want to plot
st = get_geonet_waveforms(
    event, delay=10, length=25, all_components=True)

fig = plot_picked(event, st)
fig.show()

Using matplotlib backend: Qt5Agg
Event has 204 picks


Note that GeoNet do not usually pick S-phases.  S-phases are more difficult for automatic pickers to identify correctly,
and can be difficult to pick manually. However, the additional information from S arrivals can greatly improve locations,
particularly for offshore earthquakes (with poor azimuthal coverage).

Lets zoom in on those P-picks.

In [2]:
short_st = Stream()
for pick in event.picks:
    if pick.phase_hint[0].upper() != "P":
        continue
    tr = st.select(id=pick.waveform_id.get_seed_string())
    tr = tr.slice(pick.time - 0.5, pick.time + 2).copy()
    short_st += tr
short_st.merge().sort(["starttime"])
fig = plot_picked(event, short_st)
fig.show()

From this view we can see that some arrivals are more impulsive than others.  The general rule is to **pick the first break** where
a break is a deviation from the background noise to the earthquake waveform. This will give you the arrival of the first arriving P-phase, 
which can be used to infer the hypocenter; the location that the earthquake started.  Sometimes this first break can be hard to spot
the change, either due to emmergent waveforms (common in media that have strong resonance) or low signal to noise ratios. It is normal
to estimate your pick uncertainty when making manual picks.  This pick uncertainty can be incorporated into earthquake location inversions
to weight the inversion, and to constrain the final location uncertainty.  Remember that earthquake locations are models of where an earthquake
happened, and they have some intrinsic uncertainty associated with them.

## Pick your own

I have written a little interactive plot for you to pick your own waveforms with. You will use this for the assignment as well. Bear in mind that this is a simple application for picking P and S phases. Other software are available and would be recommended for research or commercial purposes because they give you more control and options for picking and uncertainty. Some popular software packages are:
- [Seisan](http://seis.geus.net/software/seisan/seisan.html)
- [SAC](https://ds.iris.edu/files/sac-manual/)
- [Seiscomp](https://www.seiscomp3.org/)

To start with we will try picking the event we just looked at. You can compare your picks to the GeoNet picks once you are done.

In [3]:
from utilities.seismic_picker import SeismicPicker
from obspy.core.event import ResourceIdentifier

event_id = "2018p546311"

picked_event = SeismicPicker(st).pick()
# Because this event already has an ID number, you can add that in to the event
# to help you keep track of things:

picked_event.resource_id = ResourceIdentifier(id=event_id)

# You will want to save your picks after you are done. To do that use something like:
picked_event.write("{0}_picked.xml".format(event_id), format="QUAKEML")
# Note you should try and keep the event ID in the filename

Make your picks using:
	left mouse button: P
	right mouse button: S
	'a': amplitude at mouse location
	'e': end duration at mouse location - needs a P-pick to calculate duration
	P-wave polarity can be picked using the up and down arrows while hovering over a P-pick.

Picks can be deleted by hovering over them and pressing the middle mouse button
Pick made at 2018-07-22T03:14:59.711280Z
Pick made at 2018-07-22T03:15:02.130940Z
Pick made at 2018-07-22T03:15:00.638820Z
Pick made at 2018-07-22T03:15:10.035180Z
Finished picking.
Finished processing event. Returning
Returning event


You can change the event_id above and you will get a different event to pick.  Pick at least 10 and compare
your picks to the GeoNet picks.

You can re-load your picks using:

```python
from obspy import read_events

catalog = read_events("your-pickfile-here") # Replace with the file you want to read in
```